### Notebook for case-problem 4 strategy 2

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.special import gamma

In [ ]:
HU=31.74

patient1 = # Hidden 
patient2 = # Hidden
patient3 = # Hidden
patient4 = # Hidden
patient5 = # Hidden 
patient6 = # Hidden 

In [ ]:
p1_sim = np.genfromtxt('Data/blood_generated/p1q4218.69', delimiter=',')
p2_sim = np.genfromtxt('Data/blood_generated/p2q3513.35', delimiter=',')
p3_sim = np.genfromtxt('Data/blood_generated/p3q4310.52', delimiter=',')
p4_sim = np.genfromtxt('Data/blood_generated/p4q4510.85', delimiter=',')
p5_sim = np.genfromtxt('Data/blood_generated/p5q3104.34', delimiter=',')
p6_sim = np.genfromtxt('Data/blood_generated/p6q3905.67', delimiter=',')

In [ ]:
vref1 = # Hidden
vref2 = # Hidden
vref3 = # Hidden
vref4 = # Hidden
vref5 = # Hidden 
vref6 = # Hidden

In [ ]:
qref1 = # Hidden 
qref2 = # Hidden 
qref3 = # Hidden 
qref4 = # Hidden 
qref5 = # Hidden  
qref6 = # Hidden 

In [ ]:
class Neural_net(torch.nn.Module):
    def __init__(self, n_in = 1, n_out =1):
        super(Neural_net, self).__init__()

        self.tanh = torch.nn.Tanh()

        self.layer1 = torch.nn.Linear(n_in,20)
        self.layer2 = torch.nn.Linear(20,20)
        self.layer3 = torch.nn.Linear(20,20)
        self.layer_out = torch.nn.Linear(20,n_out)

    def forward(self, x):
        x = self.layer1(x)
        x = self.tanh(x)
        x = self.layer2(x)
        x = self.tanh(x)
        x = self.layer3(x)
        x = self.tanh(x)
        x = self.layer_out(x)

        return x

In [ ]:
class blood_1pinn:
    def __init__(self, epochs, data, nde, params = {}, gamma = None):
        self.epochs = epochs
        self.model = Neural_net(n_out=1)
        self.domain = torch.linspace(0,100, nde, requires_grad=True).reshape(-1,1)
        
        ###  scaling params
        self.params_unconstr ={}
        for prm, init in params.items():
            self.params_unconstr[prm] = torch.nn.Parameter(torch.tensor([init]))
            self.model.register_parameter(name=prm, param = self.params_unconstr[prm])
        
        self.const = torch.tensor([1.], dtype = torch.float)
            

        #optimizers
        self.lbfgs_optimizer = torch.optim.LBFGS(params = self.model.parameters(), lr = 0.001,max_iter = 500)
        self.adam_optimizer = torch.optim.Adam(params = self.model.parameters(), lr = 0.001)

        
        patient_data, sim_data, vref ,qref  = data

        self.vref = vref
        self.qc =self.m_inj(np.linspace(0,100,nde))
        self.Q_ref = torch.tensor(qref, dtype=torch.float, requires_grad=False).reshape(-1,1)

        #### Data        

        if gamma: ## Using a gamma variate fit of data at end-compartment instead of pure data-points
            popt, pcov = curve_fit(self.scaled_gamma, patient_data[0], patient_data[1], bounds = ((1000,1,1),(10000,10,10)))
            self.c_patient = torch.tensor(self.scaled_gamma(np.linspace(0,100,100),popt[0],popt[1], popt[2]), dtype=torch.float).reshape(-1,1)
            self.t_patient = torch.linspace(0,100,100).reshape(-1,1) 

        else:
            self.c_patient = torch.tensor(patient_data[1], dtype = torch.float).reshape(-1,1)
            self.t_patient = torch.tensor(patient_data[0], dtype=torch.float).reshape(-1,1) 




        sample_ids = np.arange(start=0, stop=len(sim_data[7]), step=len(sim_data[7])//nde)
        self.c7 = torch.tensor(sim_data[7][sample_ids], dtype=torch.float).reshape(-1,1)
        
        self.patient_data = patient_data
        
        self.cp_max = max(self.c_patient)
        self.cp_min = min(self.c_patient)
        self.c_patient_norm = (self.c_patient - self.cp_min)/(self.cp_max-self.cp_min) 


    #Constrain parameters to be in range

    @property
    def sq_param(self):
        if 'sq' in self.params_unconstr:
            return self.params_unconstr['sq']
        else:
            return self.const
    
    @property
    def sv_param(self):
        if 'sv' in self.params_unconstr:
            return self.params_unconstr['sv']
        else:
            return self.const
    
    @property
    def sc_param(self):
        if 'sc' in self.params_unconstr:
            return self.params_unconstr['sc']
        else:
            return self.const
    

    #------------- Gamma variate 
    def scaled_gamma(self, x,c, k, theta):
            frac = 1/(gamma(k)*theta**(k))
            return c*frac*x**(k-1)*np.exp(-(x/theta))

    # ------------
    def m_inj(self,t,ti=[0,3.33,3.33,6.67,6.67],qi=[6,6,6,6,0],ci=[350,350,0,0,0]):
        ti=np.array(ti)
        qi=np.array(qi)
        # ci=np.array(ci)
        # m = qi*ci
        return torch.tensor(np.interp(t,ti,qi),dtype = torch.float)


    def wrap_grad(self, f,x):
        return torch.autograd.grad(f,x,
        grad_outputs=torch.ones_like(x),
        retain_graph=True,
        create_graph=True)[0]
        

    def de_loss(self):
        c_pred = self.model(self.domain)

        c =c_pred*(self.cp_max - self.cp_min) + self.cp_min
        dup = self.wrap_grad(c_pred, self.domain)

        norm_fac = (self.cp_max -self.cp_min)

        Q = self.qc + self.Q_ref*self.sq_param
        c7 = self.c7*self.sc_param

        z = dup -(Q*c7-Q*c)/(self.vref*self.sv_param*norm_fac)
        return torch.mean(z**2) + c_pred[0]**2


    def data_loss(self):
        u7_pred = self.model(self.t_patient)
        return torch.mean((u7_pred - self.c_patient_norm)**2)
    

    def combined_loss(self):
        return self.data_loss() #self.de_loss() + self.data_loss()
    
    def model_call(self):
        return self.model(self.domain)
    
    def plot_preds(self):
        pred = self.model(self.domain)
        plt.plot(self.domain.detach(),pred.detach()*(self.cp_max -self.cp_min) + self.cp_min)
        plt.scatter(self.patient_data[0], self.patient_data[1])
        plt.grid()
        #plt.legend()
    

    def lbfgs_train(self):
            self.model.train()    
            for epoch in range(self.epochs):
                def closure():
                    self.lbfgs_optimizer.zero_grad()
                    loss = self.combined_loss()
                    loss.backward()
                    return loss
                self.lbfgs_optimizer.step(closure=closure)
                print(f'Epoch {epoch}, training loss: {self.combined_loss()}')
            self.plot_preds()
    

    def adam_train(self):
            steps = 1000
            for epoch in range(self.epochs):
                for step in range(steps):
                    def closure():
                        self.adam_optimizer.zero_grad()
                        loss = self.combined_loss()
                        loss.backward()
                        return loss
                    self.adam_optimizer.step(closure=closure)
                print(f'Epoch {epoch}, training loss: {self.combined_loss().item()} de_loss: {self.de_loss().item()}, data_loss {self.data_loss()}, sq {self.sq_param.item()} sv: {self.sv_param.item()} sc: {self.sc_param.item()} , Q: {(self.Q_ref*self.sq_param).item()}')
            self.plot_preds()

In [ ]:
test = blood_1pinn(epochs = 20, data = (patient4,p4_sim, vref4[7], qref4),params = {'sv': 1., 'sc':1.}, nde = 100, gamma=False)


In [ ]:
# Model configs:

qcv ={'sq':1., 'sv':1. , 'sc':1.}
qv = {'sq':1., 'sv':1. }
qc = {'sq':1. , 'sc':1.}
q = {'sq':1.}
vc = {'sv':1. , 'sc':1.}
c = { 'sc':1.}
v = {'sv':1.}
no_params = {}



In [ ]:
p1_data = (patient1,p1_sim, vref1[7], qref1)

p1_none = blood_1pinn(epochs = 20, data = p1_data,params =no_params, nde = 100, gamma=False)

In [ ]:
p1_none.adam_train()
# plt.gca().axes.get_yaxis().set_visible(False)
# plt.gca().axes.get_xaxis().set_visible(False)

### Patient 1

In [ ]:
p1_data = (patient1,p1_sim, vref1[7], qref1)

p1_none = blood_1pinn(epochs = 20, data = p1_data,params =no_params, nde = 100, gamma=False)
p1_none_gamma = blood_1pinn(epochs = 20, data = p1_data,params =no_params, nde = 100, gamma=True)

p1_qcv = blood_1pinn(epochs = 20, data = p1_data,params =qcv, nde = 100, gamma=False)
p1_qcv_gamma = blood_1pinn(epochs = 20, data = p1_data,params =qcv, nde = 100, gamma=True)

p1_qv = blood_1pinn(epochs = 20, data = p1_data,params =qv, nde = 100, gamma=False)
p1_qv_gamma = blood_1pinn(epochs = 20, data = p1_data,params =qv, nde = 100, gamma=True)

p1_qc = blood_1pinn(epochs = 20, data = p1_data,params =qc, nde = 100, gamma=False)
p1_qc_gamma = blood_1pinn(epochs = 20, data = p1_data,params =qc, nde = 100, gamma=True)

p1_q = blood_1pinn(epochs = 20, data = p1_data,params =q, nde = 100, gamma=False)
p1_q_gamma = blood_1pinn(epochs = 20, data = p1_data,params =q, nde = 100, gamma=True)

p1_vc = blood_1pinn(epochs = 20, data = p1_data,params =vc, nde = 100, gamma=False)
p1_vc_gamma = blood_1pinn(epochs = 20, data = p1_data,params =vc, nde = 100, gamma=True)

p1_c = blood_1pinn(epochs = 20, data = p1_data,params =c, nde = 100, gamma=False)
p1_c_gamma = blood_1pinn(epochs = 20, data = p1_data,params =c, nde = 100, gamma=True)

p1_v = blood_1pinn(epochs = 20, data = p1_data,params =v, nde = 100, gamma=False)
p1_v_gamma = blood_1pinn(epochs = 20, data = p1_data,params =v, nde = 100, gamma=True)


In [ ]:
p1_none.adam_train()

In [ ]:
p1_none_gamma.adam_train()



In [ ]:
p1_qcv.adam_train()


In [ ]:
p1_qcv_gamma.adam_train()



In [ ]:
p1_qv.adam_train()


In [ ]:
p1_qv_gamma.adam_train()

In [ ]:
p1_qc.adam_train()


In [ ]:
p1_qc_gamma.adam_train()

In [ ]:

p1_q.adam_train()


In [ ]:
p1_q_gamma.adam_train()



In [ ]:
p1_vc.adam_train()


In [ ]:
p1_vc_gamma.adam_train()

In [ ]:

p1_c.adam_train()


In [ ]:
p1_c_gamma.adam_train()



In [ ]:
p1_v.adam_train()


In [ ]:
p1_v_gamma.adam_train()